### NLP PRoject

In [1]:
import csv
import re
import random

!cat *.compare > NLP-Translation-GR-EN-Data.txt

In [2]:
input_file = 'NLP-Translation-GR-EN-Data.txt'  
greek_csv = 'Greek-Text.csv'  
english_csv = 'English-Text.csv'  

In [3]:
# Open the input file for reading
with open(input_file, 'r', encoding='utf-8') as infile:
    lines = infile.readlines()

# Open the output CSV files for writing
with open(greek_csv, 'w', encoding='utf-8', newline='') as greek_out, \
     open(english_csv, 'w', encoding='utf-8', newline='') as english_out:

    # Create CSV writers
    greek_writer = csv.writer(greek_out)
    english_writer = csv.writer(english_out)

    # Write headers to the CSV files
    greek_writer.writerow(['Sentence', 'Line'])
    english_writer.writerow(['Sentence', 'Line'])

    # Process the input file lines in the correct pattern
    greek_line_number = 1
    english_line_number_1 = 2
    english_line_number_2 = 3
    
    # Iterate over the lines and select the Greek and English lines
    for i in range(0, len(lines), 8):

        # Write Greek lines to Greek-Text CSV 
        greek_line = lines[i].strip()
        if greek_line:
            greek_writer.writerow([greek_line, greek_line_number])  # Write sentence and line number
            greek_line_number += 4  # Move to the next Greek line number (1, 5, 9, 13, ...)

        # Write English lines to English-Text CSV (2-3, 6-7, 10-11, ...)
        if i + 1 < len(lines):  # Ensure the line exists before writing
            english_line1 = lines[i + 1].strip()
            if english_line1:
                english_writer.writerow([english_line1, english_line_number_1])  # Write English sentence and line number
                english_line_number_1 += 4  # Move to the next English line number (2, 6, 10, 14, ...)

        if i + 2 < len(lines):  # Ensure the line exists before writing
            english_line2 = lines[i + 2].strip()
            if english_line2:
                english_writer.writerow([english_line2, english_line_number_2])  # Write English sentence and line number
                english_line_number_2 += 4  # Move to the next English line number (3, 7, 11, 15, ...)

print("CSV files have been written successfully.")

CSV files have been written successfully.


In [4]:
# Define file paths
input_file = 'Greek-Text.csv'  # Input CSV file containing Greek sentences with line numbers
output_csv = 'GR-Linguistics.csv'  # Output CSV file for linguistic phenomena

# Define keywords for each linguistic phenomenon
# Negation keywords (more common negation terms in Greek)
negation_keywords = [
    'Δεν', 'Μη', 'Ούτε', 'Πουθενά', 'Ποτέ', 'Καθόλου', 'Κανένας', 'Καμία', 'Κανένα', 'Τίποτα',
    'ούτε καν', 'μηδέν', 'απαγορεύεται', 'όχι', 'μηδέν'
]

# Coordination keywords (common words used for coordination in Greek)
coordination_keywords = [
    'ή', 'και', 'ή...ή', 'ούτε...ούτε', 'άρα'
]

# Subordination keywords (words used to introduce subordinate clauses)
subordination_keywords = [
    'αν', 'σαν', 'πως', 'ότι', 'όπως', 'πριν', 'για να', 'όταν', 'εφόσον', 'αν και', 'αφού', 'διότι', 'γιατί', 'επειδή'
]


# Open the input Greek text file (assuming it's in CSV format)
with open(input_file, 'r', encoding='utf-8') as infile:
    reader = csv.reader(infile)
    next(reader)  # Skip the header

    # Open the output CSV file for writing
    with open(output_csv, 'w', encoding='utf-8', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(['Sentence', 'Line', 'Linguistic Phenomenon'])

        # Track if we write any data to the output file
        rows_written = 0

        # Iterate over the rows of the Greek text CSV
        for row in reader:
            greek_sentence = row[0].strip()  # Extract the Greek sentence
            try:
                # Try to convert the second column to an integer for line number
                line_number = int(row[1])
            except ValueError:
                # If the line number is not a valid integer, skip this row
                print(f"Warning: Skipping row due to invalid line number format: {row}")
                continue

            # Check for each linguistic phenomenon using regex
            if any(re.search(rf'\b{keyword}\b', greek_sentence) for keyword in negation_keywords):
                writer.writerow([greek_sentence, line_number, 'Negation'])
                rows_written += 1
            elif any(re.search(rf'\b{keyword}\b', greek_sentence) for keyword in coordination_keywords):
                writer.writerow([greek_sentence, line_number, 'Coordination'])
                rows_written += 1
            elif any(re.search(rf'\b{keyword}\b', greek_sentence) for keyword in subordination_keywords):
                writer.writerow([greek_sentence, line_number, 'Subordination'])
                rows_written += 1

        # Check if any rows were written and provide feedback
        if rows_written > 0:
            print(f"Linguistic phenomena CSV file has been written successfully: {output_csv}")
        else:
            print("No matching linguistic phenomena were found.")


Linguistic phenomena CSV file has been written successfully: GR-Linguistics.csv


In [5]:
!sort GR-Linguistics.csv | uniq > GR-Linguistics-uid.csv

In [6]:
!grep -c "Negation" GR-Linguistics-uid.csv
!grep -c "Coordination" GR-Linguistics-uid.csv
!grep -c "Subordination" GR-Linguistics-uid.csv


1071
952
1347


In [7]:
import csv

# Read the linguistic CSV file and write sentences to source.txt
with open('GR-Linguistics-uid.csv', newline='', encoding='utf-8') as linguistics_file:
    linguistics_reader = csv.reader(linguistics_file)
    
    # Skip the header row
    next(linguistics_reader)
    
    # Open the source.txt file to write the linguistic sentences
    with open('source.txt', 'w', encoding='utf-8') as source_file:
        for row in linguistics_reader:
            sentence = row[0]  # First column contains the linguistic sentence
            source_file.write(sentence + "\n")  # Write each sentence to source.txt one below the other

# Read the English CSV file into a list for easier access
with open('English-Text.csv', newline='', encoding='utf-8') as english_file:
    english_reader = csv.reader(english_file)
    english_sentences = list(english_reader)  # Convert to list to easily access by index

# Open the target.txt file to write the English sentences based on linguistic row numbers
with open('target.txt', 'w', encoding='utf-8') as target_file:
    # Rewind the linguistics file to process again, this time for line numbers
    with open('GR-Linguistics-uid.csv', newline='', encoding='utf-8') as linguistics_file:
        linguistics_reader = csv.reader(linguistics_file)
        
        # Skip the header row again
        next(linguistics_reader)
        
        for row in linguistics_reader:
            try:
                line_number = int(row[1])  # Second column contains the line number for English sentences
                
                # Find the row in the English file where the second column equals line_number + 1
                found_row = None
                for i, eng_row in enumerate(english_sentences):
                    if str(line_number + 1) == eng_row[1]:  # Matching line_number + 1 in English CSV
                        found_row = i
                        break
                
                if found_row is not None:
                    # Write the found row and the next row (found_row + 1)
                    if found_row + 1 < len(english_sentences):  # Ensure there's a next row
                        target_file.write(english_sentences[found_row][0] + "\n")  # Sentence from found_row
                       # target_file.write(english_sentences[found_row + 1][0] + "\n")  # Sentence from next row
                    else:
                        print(f"Skipping line {line_number + 1} because there is no next row in English.")
                else:
                    print(f"Line number {line_number + 1} not found in English-Text.csv.")
            except ValueError:
                print(f"Skipping row due to invalid line number: {row[1]}")


Skipping row due to invalid line number: Line


In [8]:
!awk '!seen[$0]++' source.txt > combined.tx

In [9]:
!awk '!seen[$0]++' target.txt > neo.txt



In [10]:
!wc -l GR-Linguistics-uid.csv

3371 GR-Linguistics-uid.csv


In [11]:
!wc -l Source-Text-GR.txt

wc: Source-Text-GR.txt: No such file or directory


In [12]:
!wc -l combined.tx

2067 combined.tx


In [13]:
!grep -in "sentence" source.txt



15:Sentence


In [14]:
!sed -n '23p' target.txt


Even traditionally, though, not all Sámi have been involved in big scale reindeer husbandry, but lived from fishing, hunting and similar, having reindeer mostly as draft animals.


In [15]:
!sed -n '23p' source.txt

Ακολούθως, στα καλύτερα από αυτά τα καταλύματα μπορεί να βρει κανείς τα πιο πολυτελή είδη κρεβατιού, ίσως ένα χειροποίητο πάπλωμα ή ένα κρεβάτι αντίκα.


In [16]:
!cp combined.txt gr-input.txt


cp: cannot stat 'combined.txt': No such file or directory


In [17]:
!cp neo.txt en-human-translation.txt

In [22]:
import csv

def process_csv(input_csv1, input_csv2, output_txt):
    try:
        # Read the second CSV into a dictionary for faster lookup
        csv2_data = {}
        with open(input_csv2, mode='r') as file2:
            reader2 = csv.reader(file2)
            for row in reader2:
                if len(row) >= 2 and row[1].isdigit():  # Ensure the second column is numeric
                    csv2_data[int(row[1])] = row[0]  # {second_column: first_column}

        # Open the first CSV and output text file
        with open(input_csv1, mode='r') as file1, open(output_txt, mode='w') as output_file:
            reader1 = csv.reader(file1)
            for row in reader1:
                if len(row) >= 2 and row[1].isdigit():  # Ensure the second column is numeric
                    value_to_find = int(row[1]) + 1  # Second column value + 1
                    if value_to_find in csv2_data:
                        output_file.write(csv2_data[value_to_find] + '\n')

        print(f"Processing complete. Check the output file: {output_txt}")
    except Exception as e:
        print(f"An error occurred: {e}")
input_csv1 = "GR-Linguistics-uid.csv"  # Replace with the first CSV filename
input_csv2 = "English-Text.csv"  # Replace with the second CSV filename
output_txt = "OUTPUT.txt"  # Replace with the desired output text filename

process_csv(input_csv1, input_csv2, output_txt)


Processing complete. Check the output file: OUTPUT.txt


In [24]:
!awk '!seen[$0]++' OUTPUT.txt > deduplicated_output.txt

In [26]:
import csv

def process_csv(input_csv1, input_csv2, gr_input_txt, output_txt):
    try:
        # Load sentences from gr_input.txt into a set
        with open(gr_input_txt, mode='r') as gr_file:
            gr_sentences = set(line.strip() for line in gr_file if line.strip())

        # Read the second CSV into a dictionary for faster lookup
        csv2_data = {}
        with open(input_csv2, mode='r') as file2:
            reader2 = csv.reader(file2)
            for row in reader2:
                if len(row) >= 2 and row[1].isdigit():  # Ensure the second column is numeric
                    csv2_data[int(row[1])] = row[0]  # {second_column: first_column}

        # Open the first CSV and output text file
        with open(input_csv1, mode='r') as file1, open(output_txt, mode='w') as output_file:
            reader1 = csv.reader(file1)
            for row in reader1:
                if len(row) >= 2 and row[0] in gr_sentences:  # Match sentence in the first column
                    if row[1].isdigit():  # Ensure the second column is numeric
                        value_to_find = int(row[1]) + 1  # Second column value + 1
                        if value_to_find in csv2_data:
                            output_file.write(csv2_data[value_to_find] + '\n')  # Write matching sentence

        print(f"Processing complete. Check the output file: {output_txt}")
    except Exception as e:
        print(f"An error occurred: {e}")

# File paths
input_csv1 = "GR-Linguistics-uid.csv"  # Replace with the first CSV filename
input_csv2 = "English-Text.csv"        # Replace with the second CSV filename
gr_input_txt = "gr-input.txt"          # Replace with the Greek input text filename
output_txt = "OUTPUT.txt"              # Replace with the desired output text filename

process_csv(input_csv1, input_csv2, gr_input_txt, output_txt)


Processing complete. Check the output file: OUTPUT.txt


In [28]:
!awk '!seen[$0]++' OUTPUT.txt > deduplicated_output.txt

In [30]:
import csv

def process_csv(input_csv1, input_csv2, gr_input_txt, output_txt):
    try:
        # Load sentences from gr_input.txt into a set
        with open(gr_input_txt, mode='r') as gr_file:
            gr_sentences = set(line.strip() for line in gr_file if line.strip())

        # Read the second CSV into a dictionary for faster lookup
        csv2_data = {}
        with open(input_csv2, mode='r') as file2:
            reader2 = csv.reader(file2)
            for row in reader2:
                if len(row) >= 2 and row[1].isdigit():  # Ensure the second column is numeric
                    csv2_data[int(row[1])] = row[0]  # {second_column: first_column}

        # Open the first CSV and output text file
        with open(input_csv1, mode='r') as file1, open(output_txt, mode='w') as output_file:
            reader1 = csv.reader(file1)
            for row in reader1:
                if len(row) >= 2 and row[0] in gr_sentences:  # Match sentence in the first column
                    if row[1].isdigit():  # Ensure the second column is numeric
                        value_to_find = int(row[1]) + 1  # Second column value + 1
                        if value_to_find in csv2_data:
                            output_file.write(csv2_data[value_to_find] + '\n')  # Write matching sentence

        print(f"Processing complete. Check the output file: {output_txt}")
    except Exception as e:
        print(f"An error occurred: {e}")

# File paths
input_csv1 = "GR-Linguistics-uid.csv"  # Replace with the first CSV filename
input_csv2 = "English-Text.csv"        # Replace with the second CSV filename
gr_input_txt = "gr-input.txt"          # Replace with the Greek input text filename
output_txt = "OUTPUT.txt"              # Replace with the desired output text filename

process_csv(input_csv1, input_csv2, gr_input_txt, output_txt)


Processing complete. Check the output file: OUTPUT.txt


In [31]:
!awk '!seen[$0]++' OUTPUT.txt > deduplicated_output.txt

In [32]:
!awk '!seen[$0]++' GR-Linguistics-uid.csv > deduplicated_output.txt

In [34]:
import csv

def copy_first_column_to_txt(input_csv, output_txt):
    try:
        with open(input_csv, mode='r', encoding='utf-8') as csv_file, open(output_txt, mode='w', encoding='utf-8') as txt_file:
            reader = csv.reader(csv_file)
            for row in reader:
                if len(row) > 0:  # Ensure there's at least one column
                    txt_file.write(row[0] + '\n')  # Write the first column to the text file
        print(f"Content of the first column copied to {output_txt}")
    except Exception as e:
        print(f"An error occurred: {e}")

# File paths
input_csv = "GR-Linguistics-uid.csv"  # Replace with your CSV filename
output_txt = "elala.txt"  # Replace with your desired text filename

copy_first_column_to_txt(input_csv, output_txt)


Content of the first column copied to elala.txt


In [35]:
!awk '!seen[$0]++' elala.txt > deduplicated_output.txt

In [36]:
import csv

def process_csv(input_csv, output_csv):
    try:
        unique_sentences = {}
        
        # Read the input CSV
        with open(input_csv, mode='r') as infile:
            reader = csv.reader(infile)
            header = next(reader, None)  # Capture the header, if present
            
            for row in reader:
                if len(row) >= 2:  # Ensure there are at least two columns
                    sentence = row[0].strip()
                    value = row[1].strip()
                    
                    # Keep only one instance of each sentence
                    if sentence not in unique_sentences:
                        unique_sentences[sentence] = value
        
        # Write to the output CSV
        with open(output_csv, mode='w', newline='') as outfile:
            writer = csv.writer(outfile)
            
            # Write the header, if the input CSV had one
            if header:
                writer.writerow(header)
            
            for sentence, value in unique_sentences.items():
                writer.writerow([sentence, value])
        
        print(f"Processing complete. Unique sentences written to {output_csv}")
    
    except Exception as e:
        print(f"An error occurred: {e}")

# File paths
input_csv = "GR-Linguistics-uid.csv"    # Replace with your input CSV filename
output_csv = "output.csv"  # Replace with your desired output CSV filename

process_csv(input_csv, output_csv)


Processing complete. Unique sentences written to output.csv


In [37]:
import csv

def find_and_write(input_csv, lookup_csv, output_txt):
    try:
        # Read the lookup CSV into a dictionary for faster access
        lookup_data = {}
        with open(lookup_csv, mode='r') as lookup_file:
            reader = csv.reader(lookup_file)
            for row in reader:
                if len(row) >= 2 and row[1].isdigit():  # Ensure the second column is numeric
                    lookup_data[int(row[1])] = row[0]  # {second_column: first_column}

        # Open the input CSV and output text file
        with open(input_csv, mode='r') as input_file, open(output_txt, mode='w') as output_file:
            reader = csv.reader(input_file)
            for row in reader:
                if len(row) >= 2 and row[1].isdigit():  # Ensure the second column is numeric
                    value_to_find = int(row[1]) + 1  # Second column value + 1
                    if value_to_find in lookup_data:
                        output_file.write(lookup_data[value_to_find] + '\n')  # Write matching content to file

        print(f"Processing complete. Check the output file: {output_txt}")
    except Exception as e:
        print(f"An error occurred: {e}")

# File paths
input_csv = "output.csv"        # Replace with the input CSV filename
lookup_csv = "English-Text.csv"        # Replace with the lookup CSV filename
output_txt = "TRANS.txt"       # Replace with the desired text output filename

find_and_write(input_csv, lookup_csv, output_txt)


Processing complete. Check the output file: TRANS.txt


In [39]:
!sed -n '999p' TRANS.txt

During the 18th century Cambodia found itself squeezed between two powerful neighbors, Thailand and Vietnam.
